In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from src.dataset import ChestXrayDataSet, CLASS_NAMES
from src.model import DenseNet121
from src.utils import compute_AUCs, compute_score_with_logits, tile
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Autoreload modules so that changes to src automatically reflect
%load_ext autoreload
%autoreload 2

In [2]:
labels = pd.read_csv("data/labels/cleaned.csv")
size = 10000 # only using 10k out of 30k cleaned dataset
X, Y = labels.iloc[:size, 0], labels.iloc[:size, 1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42, shuffle=True)

In [3]:
N_CLASSES = len(CLASS_NAMES)
BATCH_SIZE = 16
DATA_DIR = "data/images"

In [4]:
normalize = transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])

train_dataset = ChestXrayDataSet(data_dir=DATA_DIR, X = X_train, Y = Y_train,
                          transform=transforms.Compose([
                              transforms.Resize(256),
                              transforms.TenCrop(224),
                              transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                              transforms.Lambda(lambda crops: torch.stack([normalize(crop) for crop in crops]))
                              ]))

# note that workers take up some amount of VRAM   
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

test_dataset = ChestXrayDataSet(data_dir=DATA_DIR, X = X_test, Y = Y_test,
                          transform=transforms.Compose([
                              transforms.Resize(256),
                              transforms.TenCrop(224),
                              transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                              transforms.Lambda(lambda crops: torch.stack([normalize(crop) for crop in crops]))
                              ]))
    
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [5]:
training = True # Flip to false to simply load pre-trained model
CKPT_PATH = "model.pth.tar" # Starter model from https://github.com/arnoweng/CheXNet
CKPT_TRAINED_PATH = "model-trained.pth" # Model trained on top of ^
cudnn.benchmark = True # Fixed input size, enables tuning for optimal use

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# initialize and load the model
model = DenseNet121(N_CLASSES).to(device)
model = torch.nn.DataParallel(model).to(device)

if training:
    if os.path.isfile(CKPT_PATH):
        print("=> loading checkpoint")
        checkpoint = torch.load(CKPT_PATH)
        model.load_state_dict(checkpoint['state_dict'], strict=False)
        print("=> loaded checkpoint")
    else:
        print("=> no checkpoint found")
else:
    if os.path.isfile(CKPT_TRAINED_PATH):
        print("=> loading checkpoint")
        checkpoint = torch.load(CKPT_TRAINED_PATH)
        # Load directly into the module else the model gets screwed up
        # https://discuss.pytorch.org/t/solved-keyerror-unexpected-key-module-encoder-embedding-weight-in-state-dict/1686/15
        model.module.load_state_dict(checkpoint, strict=True)
        print("=> loaded checkpoint")
    else:
        print("=> no checkpoint found")

=> loading checkpoint
=> loaded checkpoint


## Training

In [6]:
# Source: https://github.com/thibaultwillmann/CheXNet-Pytorch/blob/master/CheXnet.ipynb
# use_amp = True
# scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

if training:
    model.train()

    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # RMSprop, Adam

    for epoch in range(20):  # loop over the dataset multiple times

        running_loss = 0.0
        correct = 0
        total = 0 
        for i, (images, labels) in enumerate(train_loader, 0): # get the inputs; data is a list of [images, labels]

            # images.shape -> [N, 10, 3, 224, 224]
            # labels.shape -> [N, 15]

            # zero the parameter gradients
            optimizer.zero_grad(set_to_none=True)

            images = images.cuda()

            # format input
            n_batches, n_crops, channels, height, width = images.size()
            image_batch = torch.autograd.Variable(images.view(-1, channels, height, width)) # 10N images: N batches contain 10 crops each decomposed into 10N images

            labels = tile(labels, 0, 10).cuda() # duplicate for each crop the label [1,2],[3,4] => [1,2],[1,2],[3,4],[3,4] -> 10N labels

            # forward + backward + optimize
            # TODO: possibly use torch.cuda.amp?            
            outputs = model(image_batch)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            correct += compute_score_with_logits(outputs, labels).sum()
            total += labels.size(0)

        print('Epoch: %d, loss: %.3f, Accuracy: %.3f' %
            (epoch + 1, running_loss, 100 * correct / total))

    print('Finished Training')

Epoch: 1, loss: 77.849, Accuracy: 35.882
Epoch: 2, loss: 69.809, Accuracy: 43.637
Epoch: 3, loss: 64.373, Accuracy: 48.875
Epoch: 4, loss: 58.172, Accuracy: 54.997
Epoch: 5, loss: 51.127, Accuracy: 61.718
Epoch: 6, loss: 44.577, Accuracy: 67.552
Epoch: 7, loss: 41.584, Accuracy: 70.050
Epoch: 8, loss: 36.365, Accuracy: 74.603
Epoch: 9, loss: 31.597, Accuracy: 78.473
Epoch: 10, loss: 27.511, Accuracy: 82.070
Epoch: 11, loss: 22.792, Accuracy: 86.053
Epoch: 12, loss: 19.135, Accuracy: 88.822
Epoch: 13, loss: 16.369, Accuracy: 90.902
Epoch: 14, loss: 13.373, Accuracy: 93.155
Epoch: 15, loss: 10.329, Accuracy: 95.245
Epoch: 16, loss: 8.423, Accuracy: 96.185
Epoch: 17, loss: 6.717, Accuracy: 97.217
Epoch: 18, loss: 5.424, Accuracy: 97.830
Epoch: 19, loss: 4.028, Accuracy: 98.590
Epoch: 20, loss: 2.796, Accuracy: 99.265
Finished Training


## Evaluation of Trained Model

In [7]:
# initialize the ground truth and output tensor
gt = torch.FloatTensor().to(device)
pred = torch.FloatTensor().to(device)

# switch to evaluate mode
model.eval()

for i, (inp, target) in enumerate(test_loader):
    target = target.cuda()
    gt = torch.cat((gt, target), 0)
    bs, n_crops, c, h, w = inp.size()
    with torch.no_grad():
        input_var = torch.autograd.Variable(inp.view(-1, c, h, w).cuda())
        # output = torch.sigmoid(model(input_var))
        output = model(input_var)
        output_mean = output.view(bs, n_crops, -1).mean(1)
        pred = torch.cat((pred, output_mean.data), 0)

In [8]:
AUROCs = compute_AUCs(gt, pred, N_CLASSES)
AUROC_avg = np.array(AUROCs).mean()
print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
for i in range(N_CLASSES):
  print('The AUROC of {} is {}'.format(CLASS_NAMES[i], AUROCs[i]))

The average AUROC is 0.763
The AUROC of Atelectasis is 0.7893153167636008
The AUROC of Cardiomegaly is 0.9275282030620466
The AUROC of Effusion is 0.8232625830343376
The AUROC of Infiltration is 0.7210117218229494
The AUROC of Mass is 0.7808354015878997
The AUROC of Nodule is 0.7699040625942075
The AUROC of Pneumonia is 0.5561091490077363
The AUROC of Pneumothorax is 0.8034585436385764
The AUROC of Consolidation is 0.6646448077581699
The AUROC of Edema is 0.777660759493671
The AUROC of Emphysema is 0.819904729692705
The AUROC of Fibrosis is 0.7708130023028051
The AUROC of Pleural_Thickening is 0.7091529809976412
The AUROC of Hernia is 0.7686673363468944


In [9]:
if training:
    torch.save(model.module.state_dict(), "model-trained.pth")